In [1]:
from transformers import AutoProcessor, SeamlessM4TModel
import torch
model_cache_dir = "../nllb_models"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
processor = AutoProcessor.from_pretrained("ylacombe/hf-seamless-m4t-large", cache_dir = model_cache_dir)
model = SeamlessM4TModel.from_pretrained("ylacombe/hf-seamless-m4t-large", cache_dir = model_cache_dir).to(device)

/home/abderrahman.skiredj/.conda/envs/skiredj/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-08 13:10:59.891825: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-08 13:10:59.929802: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 13:10:59.929834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 13:10:59.931508: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register facto

In [2]:
source_language_code = "ary"
target_language_code="eng"

In [3]:
def translate_single(input_text):
    input_data = processor(text = input_text, src_lang=source_language_code, return_tensors="pt").to(device)
    tokens_ids = model.generate(**input_data, generate_speech=False, tgt_lang=target_language_code, num_beams=5, do_sample=True)[0].cpu().squeeze().detach().tolist()
    text_out = processor.decode(tokens_ids, skip_special_tokens=True)
    return text_out

In [4]:
def translate_batch(input_texts):
    input_data = processor(text = input_texts, src_lang=source_language_code, return_tensors="pt").to(device)
    tokens_ids = model.generate(**input_data, generate_speech=False, tgt_lang=target_language_code, num_beams=5, do_sample=True)[0]
    decoded_texts = [processor.decode(seq, skip_special_tokens=True) for seq in tokens_ids]
    return decoded_texts

In [35]:
input_texts = ["عافاك كيندير نخلص بلاكارط؟", "أفين أساط اش خبارك؟", "تبلعاتلي لكارط في لكيشي", "بغيت نسد لكونط"]
translate_batch(input_texts)

['Can I pay by credit card, please?',
 'Where can I find you?',
 "I'm going to get a ticket to Kishi.",
 'I want to go to the county.']

In [5]:
import pandas as pd
df = pd.read_excel('test_set_darija_8fev24.xlsx')
df

,text,label,language
0,هادشي واعر,general_positive_feedback,ar
1,كيفاش نصلح طابعة عالقة؟,oodoos,ar
2,واش نقدر نلغي طلبية لشهادة ايداع جديدة؟,cancel_order,ar
3,عمليات مشبوهة فحسابي.,compromised_card,ar
4,تأجل الماتش، شنو لازم ندير؟,get_refund,ar
...,...,...,...
357,تقدر تسول الوكيل على ساعات دعم الزبائن؟,contact_customer_service,ar
358,كيفاش نعرف بلي تأميني مازال ساري؟,insurance,ar
359,بغيت نعدل المعلومات فبروفيل ديالي، كيفاش نديرها؟,edit_account,ar
360,تقدر تحاول تواجه مشاكلي بلا ما تعطيني جوابات ج...,general_negative_feedback,ar


In [6]:
df

,text,label,language
0,هادشي واعر,general_positive_feedback,ar
1,كيفاش نصلح طابعة عالقة؟,oodoos,ar
2,واش نقدر نلغي طلبية لشهادة ايداع جديدة؟,cancel_order,ar
3,عمليات مشبوهة فحسابي.,compromised_card,ar
4,تأجل الماتش، شنو لازم ندير؟,get_refund,ar
...,...,...,...
357,تقدر تسول الوكيل على ساعات دعم الزبائن؟,contact_customer_service,ar
358,كيفاش نعرف بلي تأميني مازال ساري؟,insurance,ar
359,بغيت نعدل المعلومات فبروفيل ديالي، كيفاش نديرها؟,edit_account,ar
360,تقدر تحاول تواجه مشاكلي بلا ما تعطيني جوابات ج...,general_negative_feedback,ar


In [7]:
texts = df['text'].tolist()

In [8]:
len(texts)

362

In [9]:
batched = [texts[k:k+32] for k in range(0, len(texts), 32)]
len(batched)

12

In [10]:
len(batched[-1])

10

In [12]:
from tqdm import tqdm
translated = []
for x in tqdm(batched):
    translated.extend(translate_batch(x))
len(translated)

100%|██████████| 12/12 [00:26<00:00,  2.19s/it]


362

In [13]:
translated

['This is naked.',
 'How do I fix a stuck printer?',
 'Can I cancel an order for a new deposit certificate?',
 'There are suspicious transactions on my account.',
 'The flight has been postponed. What should I do?',
 "What's the best way to train for a marathon?",
 'Can I cancel an order for a debit card rejection fee at the point of sale?',
 'I have a question. What should I do to avoid losing customer service?',
 'Ask the agent where we can get a refund.',
 'I need information. What should I do to contact customer service?',
 'What should I do if my card is withdrawn from the machine during working hours?',
 'Help me get the card fees.',
 "The ATM got stuck on me and I didn't know what to do.",
 'Can you tell me what the e-mail address of the customer service is?',
 'I bought something and I want to change it.',
 'Help me close my account.',
 'Someone made a copy of my card! I still have mine but I have to withdraw 500 pounds in cash, help me!',
 'What kind of medical expenses are el

In [14]:
df['translated'] = translated
df

,text,label,language,translated
0,هادشي واعر,general_positive_feedback,ar,This is naked.
1,كيفاش نصلح طابعة عالقة؟,oodoos,ar,How do I fix a stuck printer?
2,واش نقدر نلغي طلبية لشهادة ايداع جديدة؟,cancel_order,ar,Can I cancel an order for a new deposit certif...
3,عمليات مشبوهة فحسابي.,compromised_card,ar,There are suspicious transactions on my account.
4,تأجل الماتش، شنو لازم ندير؟,get_refund,ar,The flight has been postponed. What should I do?
...,...,...,...,...
357,تقدر تسول الوكيل على ساعات دعم الزبائن؟,contact_customer_service,ar,Can you ask the agent about the customer servi...
358,كيفاش نعرف بلي تأميني مازال ساري؟,insurance,ar,How do I know if my insurance is still valid?
359,بغيت نعدل المعلومات فبروفيل ديالي، كيفاش نديرها؟,edit_account,ar,I want to change the information in my profile...
360,تقدر تحاول تواجه مشاكلي بلا ما تعطيني جوابات ج...,general_negative_feedback,ar,Can you try to solve my problems without givin...


In [15]:
df.to_excel('test_set_darija_8fev24_translated_eng.xlsx', index=False)

# Draft

In [24]:
input_text = "عافاك كيندير نخلص بلاكارط؟"
source_language_code = "ary"
input_data = processor(text = input_text, src_lang=source_language_code, return_tensors="pt").to(device)

In [25]:
input_data

{'input_ids': tensor([[256004,    282,   2838, 178216, 207372, 136297,    208, 130999,  36307,
          37676, 248026, 248760,      3,      0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]], device='cuda:0')}

In [26]:
input_data = processor(text = 5*[input_text], src_lang=source_language_code, return_tensors="pt").to(device)

In [27]:
input_data['input_ids'].shape

torch.Size([5, 14])

In [12]:
target_language_code="eng"
tokens_ids = model.generate(**input_data, generate_speech=False, tgt_lang=target_language_code, num_beams=5, do_sample=True)[0].cpu().squeeze().detach().tolist()

In [28]:
tokens_ids = model.generate(**input_data, generate_speech=False, tgt_lang=target_language_code, num_beams=5, do_sample=True)[0]
tokens_ids

tensor([[     3, 256022,   4791,    117,  10802,    983,  64473,  35120,    708,
         122777,    698,      3],
        [     3, 256022,   4791,    117,  10802,    983,  64473,  35120,    708,
         122777,    698,      3],
        [     3, 256022,   4791,    117,  10802,    983,  64473,  35120,    708,
         122777,    698,      3],
        [     3, 256022,   4791,    117,  10802,    983,  64473,  35120,    708,
         122777,    698,      3],
        [     3, 256022,   4791,    117,  10802,    983,  64473,  35120,    708,
         122777,    698,      3]], device='cuda:0')

In [29]:
tokens_ids.shape

torch.Size([5, 12])

In [31]:
# Assuming tokens_ids has the shape [5, 12], indicating 5 sequences
decoded_texts = [processor.decode(seq, skip_special_tokens=True) for seq in tokens_ids]
decoded_texts

['Can I pay by credit card, please?',
 'Can I pay by credit card, please?',
 'Can I pay by credit card, please?',
 'Can I pay by credit card, please?',
 'Can I pay by credit card, please?']

In [13]:
text_out = processor.decode(tokens_ids, skip_special_tokens=True)
text_out

TypeError: argument 'ids': 'list' object cannot be interpreted as an integer